Import Sentiment related modules

In [4]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Get the current working directory
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# Check if we need to change directory to the project root
if os.path.basename(current_dir) == 'scripts':
    project_root = os.path.dirname(current_dir)
    os.chdir(project_root)
    print(f"Changed working directory to: {project_root}")
    sys.path.append(current_dir)
else:
    scripts_path = os.path.join(current_dir, 'scripts')
    if os.path.exists(scripts_path):
        sys.path.append(scripts_path)

# Import modules
try:
    from sentiment_utils import apply_bert_sentiment, aggregate_sentiment
except ImportError as e:
    # Fallback logic if imports fail directly
    sys.path.append(os.getcwd())
    from sentiment_utils import apply_bert_sentiment, aggregate_sentiment


Current working directory: c:\Users\mihret.mekbebe\Documents\REQUESTS\KAIM\Customer-Experience-Analytics-for-Fintech-Apps-Week2


c:\Users\mihret.mekbebe\Documents\REQUESTS\KAIM\Customer-Experience-Analytics-for-Fintech-Apps-Week2\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed. Error loading "c:\Users\mihret.mekbebe\Documents\REQUESTS\KAIM\Customer-Experience-Analytics-for-Fintech-Apps-Week2\venv\Lib\site-packages\torch\lib\c10.dll" or one of its dependencies.